**Aşama 1: Veri Hazırlığı ve Ön İşleme**

*İlk aşamada, log dosyalarını oluşturup, bu verileri uygun bir formata getireceğiz.*

1.1. Log Dosyasının Oluşturulması

Öncelikle, örnek bir web trafik log dosyası oluşturmamız gerekiyor. Bu dosya, bir web sunucusundan gelen tipik HTTP isteklerini temsil edecek. 

In [3]:
import random
import datetime

# Örnek IP adresleri ve sayfa yolları
ip_addresses = ["192.168.1.1", "10.0.0.1", "172.16.0.1"]
pages = ["/home", "/about", "/contact", "/products", "/cart"]
http_methods = ["GET", "POST", "PUT", "DELETE"]
status_codes = ["200", "400", "404", "500"]

# Log dosyasının oluşturulması
with open("web_traffic.log", "w") as f:
    for _ in range(1000):  # 1000 satırlık log oluştur
        ip = random.choice(ip_addresses)
        page = random.choice(pages)
        method = random.choice(http_methods)
        status_code = random.choice(status_codes)
        timestamp = datetime.datetime.now().strftime('%d/%b/%Y:%H:%M:%S %z')
        log_entry = f'{ip} - - [{timestamp}] "{method} {page} HTTP/1.1" {status_code} -\n'
        f.write(log_entry)

print("Log dosyası oluşturuldu.")

Log dosyası oluşturuldu.


1.2. Log Verilerinin Seçilmesi ve Temizlenmesi

Oluşturulan log dosyasını analiz edip, gerekli verileri seçmemiz gerekiyor. Örneğin, IP adresleri, erişilen sayfalar, ve zaman damgaları gibi verileri ayıklayabiliriz. Bu veriler daha sonra bir vektör veri tabanına yüklenmek üzere hazırlanacak.

In [37]:
import re

# Log dosyasını okuma ve gerekli verileri ayıklama
log_data = []
with open("web_traffic.log", "r") as f:
    for line in f:
        match = re.match(r'(\d+\.\d+\.\d+\.\d+).*?\[(.*?)\] "(.*?) (.*?) HTTP', line)
        if match:
            ip, timestamp, method, page = match.groups()
            log_data.append({"ip": ip, "timestamp": timestamp, "method": method, "page": page})

print(f"{len(log_data)} satır log verisi ayıklandı.")

for entry in log_data:
    print(entry)

1000 satır log verisi ayıklandı.
{'ip': '192.168.1.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'GET', 'page': '/cart'}
{'ip': '10.0.0.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'DELETE', 'page': '/about'}
{'ip': '172.16.0.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'DELETE', 'page': '/about'}
{'ip': '10.0.0.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'POST', 'page': '/about'}
{'ip': '172.16.0.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'GET', 'page': '/about'}
{'ip': '192.168.1.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'GET', 'page': '/about'}
{'ip': '10.0.0.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'DELETE', 'page': '/about'}
{'ip': '10.0.0.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'DELETE', 'page': '/home'}
{'ip': '10.0.0.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'PUT', 'page': '/contact'}
{'ip': '10.0.0.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'POST', 'page': '/home'}
{'ip': '172.16.0.1', 'ti

1.3. Vektörlere Dönüştürme ve Vektör Veri Tabanına Yükleme

Verileri vektörlere dönüştürmek için öncelikle bunları metin tabanlı bir temsile dönüştüreceğiz. Daha sonra bu vektörleri FAISS vektör veri tabanına yükleyeceğiz.

Öncelikle, sentence-trasformer kütüphanesini yükleyelim. Ardından metin temsili oluşturmak için basit bir sayfa yolu ve IP adresi birleştirmesi yapalım:

In [12]:
!pip install sentence-transformers

In [14]:
from sentence_transformers import SentenceTransformer

# SentenceTransformer modeli
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Vektör veritabanına yüklenecek metinleri oluşturma
texts = [f"{entry['ip']} {entry['page']}" for entry in log_data]

# Metinleri vektörlere dönüştürme
vectors = model.encode(texts)

print(f"{len(vectors)} adet vektör oluşturuldu.")

C:\Users\edana\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\edana\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1000 adet vektör oluşturuldu.


Sonraki adımda, bu vektörleri FAISS vektör veri tabanına yükleyeceğiz:

In [34]:
import faiss

# FAISS index oluşturma ve vektörleri ekleme
dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 mesafe metrikli FAISS index
index.add(vectors)

print("Vektör veritabanı oluşturuldu ve veriler yüklendi.")

Vektör veritabanı oluşturuldu ve veriler yüklendi.


**Aşama 2: RAG Modelinin Kurulumu**

*Bu aşamada, RAG modelini kuracağız. RAG modelinin bilgi alma kısmında FAISS indexini kullanacağız, jeneratif model kısmında ise GPT dil modeli kullanılacak.*

2.1. Bilgi Alma (Retrieval)

Kullanıcıdan gelen sorulara uygun log verilerini bulmak için FAISS indexini kullanalım.

In [42]:
def retrieve_logs(query, top_k=5):
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, top_k)
    return [log_data[idx] for idx in indices[0]]

# Örnek sorgu oluşturma
# 192.168.1.1 hakkında sayfası için bilgi getirme
query = "192.168.1.1 hakkında bilgi"
retrieved_logs = retrieve_logs(query)
print(f"Sorgu için bulunan loglar: {retrieved_logs}")

Sorgu için bulunan loglar: [{'ip': '192.168.1.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'GET', 'page': '/about'}, {'ip': '192.168.1.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'GET', 'page': '/about'}, {'ip': '192.168.1.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'POST', 'page': '/about'}, {'ip': '192.168.1.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'POST', 'page': '/about'}, {'ip': '192.168.1.1', 'timestamp': '15/Aug/2024:08:48:51 ', 'method': 'PUT', 'page': '/about'}]


2.2. Jeneratif Model (Generation) 

Jeneratif model, bulunan log verilerini kullanarak kullanıcıya bir yanıt oluşturacak

In [44]:
from transformers import pipeline

# Jeneratif model oluşturma
generator = pipeline('text-generation', model='gpt2')

def generate_response(retrieved_logs):
    context = " ".join([f"{log['ip']} {log['page']}" for log in retrieved_logs])
    response = generator(f"Kullanıcı sorusu: {query}. Loglardan elde edilen bilgiler: {context}", max_length=150)
    return response[0]['generated_text']

# Yanıt oluşturma
response = generate_response(retrieved_logs)
print(f"Yanıt: {response}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Yanıt: Kullanıcı sorusu: 192.168.1.1 hakkında bilgi. Loglardan elde edilen bilgiler: 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.0/8192 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.


**Aşama 3: Sistem Entegrasyonu ve Test**

*Bilgi alma ve jeneratif modeli birleştirerek entegre bir sistem oluşturacağız.*

In [39]:
def answer_question(query):
    retrieved_logs = retrieve_logs(query)
    response = generate_response(retrieved_logs)
    return response

# Test
# 172.16.0.1 hakkında sayfası için bilgi getirme
query = "172.16.0.1 sayfası hakkında bilgi"
print(f"Soru: {query}")
print(f"Cevap: {answer_question(query)}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Soru: 172.16.0.1 sayfası hakkında bilgi
Cevap: Kullanıcı sorusu: 172.16.0.1 sayfası hakkında bilgi. Loglardan elde edilen bilgiler: 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 173.8.0.1 /about 173.8.0.1 /about 173.8.0.1 /about 174.2.0.1 makealınc: 172.16.0.1


**Aşama 4: Performans Değerlendirmesi**

*Bu aşamada, sistemin doğruluğunu ve performansını değerlendireceğiz. Yanıtların kalitesini artırmak için yapabileceğimiz iyileştirmeleri düşüneceğiz.*

In [30]:
# Performans değerlendirme için test soruları
test_queries = [
    "192.168.1.1 hakkında sayfası",
    "10.0.0.1 hakkında sayfası",
    "172.16.0.1 hakkında sayfası"
]

for query in test_queries:
    print(f"Soru: {query}")
    print(f"Cevap: {answer_question(query)}\n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Soru: 192.168.1.1 hakkında sayfası
Cevap: Kullanıcı sorusu: 192.168.1.1 hakkında sayfası. Loglardan elde edilen bilgiler: 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168.1.1 /about 192.168

Soru: 10.0.0.1 hakkında sayfası


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Cevap: Kullanıcı sorusu: 10.0.0.1 hakkında sayfası. Loglardan elde edilen bilgiler: 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0.0.1 /about 10.0

Soru: 172.16.0.1 hakkında sayfası
Cevap: Kullanıcı sorusu: 172.16.0.1 hakkında sayfası. Loglardan elde edilen bilgiler: 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16.0.1 /about 172.16



**İyileştirme önerileri:**
- Daha büyük ve özelleştirilmiş bir dil modeli kullanarak daha doğru yanıtlar oluşturulabilir.
- Log verilerinin ön işlemesi geliştirilerek, daha anlamlı vektörler elde edilebilir.

In [28]:
!pip install --upgrade transformers huggingface_hub